# Module 3.1: Construct Dynamic Circuits  

> Task 3.1 from the IBM Qiskit v2.x Study Guide.  
> Sub-topics: `if_test`, `if_else`, `for_loop`, `while_loop`.  

**References**  
- IBM Docs: Classical Feedforward & Control Flow  
  https://quantum.cloud.ibm.com/docs/en/guides/classical-feedforward-and-control-flow  
- Fake backend choice: `FakeFractionalBackend` supports `if_else` and `while_loop`.

In [1]:
# Metadata: print versions at runtime.
import qiskit, sys
from qiskit_ibm_runtime import __version__ as ibm_runtime_version
print("Python :", sys.version.split()[0])
print("Qiskit  :", qiskit.__version__)
print("qiskit-ibm-runtime :", ibm_runtime_version)

# Minimal imports used across this notebook
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, generate_preset_pass_manager
from qiskit_ibm_runtime.fake_provider import FakeFractionalBackend
from qiskit_ibm_runtime import SamplerV2
import numpy as np
rng = np.random.default_rng(12345)

# Helper to transpile with preset PM and run safely.
backend = FakeFractionalBackend()

def pm_run(qc, level=1):
    pm = generate_preset_pass_manager(optimization_level=level, backend=backend)
    return pm.run(qc)

def run_and_print(tqc):
    sampler = SamplerV2(mode=backend)  # v2.x API
    res = sampler.run([tqc]).result()
    try:
        first = res[0]
        if hasattr(first, "data") and hasattr(first.data, "meas"):
            try:
                dist = first.data.meas.get_dist()
                print("Distribution:", dict(dist))
            except Exception:
                print("First element data:", first.data)
        else:
            print("First element:", first)
    except Exception:
        print(res)
    return res

Python : 3.12.9
Qiskit  : 2.2.1
qiskit-ibm-runtime : 0.41.1


## 1) Conceptual Overview  

Dynamic circuits allow real-time classical control within a quantum program.  
They enable mid-circuit measurement and conditional branching.  

**Key ideas**  
- *Classical feedforward:* Measure a qubit and use the result to control later ops.  
- *Control flow:* Use constructs like `if_test`, `if_else`, `for_loop`, and  
  `while_loop` to branch and iterate at runtime.  
- *Execution model:* Qiskit lowers control flow into OpenQASM 3 with classical  
  conditions evaluated on device.

## 2) Hands-on Examples (baseline)  

We use a fake backend that supports dynamic circuits to build and run examples  
via `SamplerV2`. We always transpile with  
`generate_preset_pass_manager(backend=...)` before `run`.

In [2]:
qc0 = QuantumCircuit(1, 1, name="baseline")
qc0.h(0)
qc0.measure(0, 0)

tqc0 = pm_run(qc0)
res0 = run_and_print(tqc0)
print(qc0.draw(output="text"))

First element: SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=1024, num_bits=1>)), metadata={'shots': 1024, 'circuit_metadata': {}})
     ┌───┐┌─┐
  q: ┤ H ├┤M├
     └───┘└╥┘
c: 1/══════╩═
           0 


## 3) Focused Exam-style Skills (Task 3.1)  

Each sub-module introduces one control-flow method with:  
- A short explanation and doc links.  
- A minimal executable example.  
- A quick exercise.

### 3.1.0  `if_test` — measured condition controls a block  

**Docs**  
- https://quantum.cloud.ibm.com/docs/en/guides/classical-feedforward-and-control-flow  

**Idea**  
Measure qubit $q_0$ into classical $c_0$, then if $c_0=1$ apply an $X$ to $q_1$.

In [3]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

q = QuantumRegister(2, "q")
c = ClassicalRegister(2, "c")
qc_if = QuantumCircuit(q, c, name="if_test_demo")

qc_if.h(q[0])               # Put q0 into superposition
qc_if.measure(q[0], c[0])   # Mid-circuit measurement

with qc_if.if_test((c[0], 1)):
    qc_if.x(q[1])           # Apply X to q1 only if c0 == 1

qc_if.measure(q[1], c[1])

tqc_if = pm_run(qc_if)
res_if = run_and_print(tqc_if)
print(qc_if.draw(output="text"))

First element: SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=1024, num_bits=2>)), metadata={'shots': 1024, 'circuit_metadata': {}})
     ┌───┐┌─┐                             
q_0: ┤ H ├┤M├─────────────────────────────
     └───┘└╥┘  ┌──────  ┌───┐ ───────┐ ┌─┐
q_1: ──────╫───┤ If-0  ─┤ X ├  End-0 ├─┤M├
           ║   └──╥───  └───┘ ───────┘ └╥┘
           ║ ┌────╨────┐                ║ 
c: 2/══════╩═╡ c_0=0x1 ╞════════════════╩═
           0 └─────────┘                1 


**Exercise**  
Change the first gate to `rx(np.pi/3, q[0])` and predict whether the probability  
of measuring `c0=1` increases or decreases compared to `h(q0)`. Then run.

### 3.1.1  `if_else` — two-way branching  

**Docs**  
- https://quantum.cloud.ibm.com/docs/en/guides/classical-feedforward-and-control-flow  

**Idea**  
Measure $q_0$ into $c_0$. If $c_0=1$ apply $H$ to $q_1$, else apply $X$.

In [4]:
q = QuantumRegister(2, "q")
c = ClassicalRegister(2, "c")
qc_ie = QuantumCircuit(q, c, name="if_else_demo")

qc_ie.h(q[0])
qc_ie.measure(q[0], c[0])

with qc_ie.if_test((c[0], 1)) as else_:
    qc_ie.h(q[1])
with else_:
    qc_ie.x(q[1])

qc_ie.measure(q[1], c[1])

tqc_ie = pm_run(qc_ie)
res_ie = run_and_print(tqc_ie)
print(qc_ie.draw(output="text"))

First element: SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=1024, num_bits=2>)), metadata={'shots': 1024, 'circuit_metadata': {}})
     ┌───┐┌─┐                                            
q_0: ┤ H ├┤M├────────────────────────────────────────────
     └───┘└╥┘  ┌──────  ┌───┐┌──────── ┌───┐ ───────┐ ┌─┐
q_1: ──────╫───┤ If-0  ─┤ H ├┤ Else-0  ┤ X ├  End-0 ├─┤M├
           ║   └──╥───  └───┘└──────── └───┘ ───────┘ └╥┘
           ║ ┌────╨────┐                               ║ 
c: 2/══════╩═╡ c_0=0x1 ╞═══════════════════════════════╩═
           0 └─────────┘                               1 


**Exercise**  
Swap the operations in the branches (`H` and `X`). Run again and compare the  
distribution of `c1`.

### 3.1.2  `for_loop` — static repetition with runtime semantics  

**Docs**  
- https://quantum.cloud.ibm.com/docs/en/guides/classical-feedforward-and-control-flow  

**Idea**  
Apply $R_x(\theta)$ three times to $q_0$ using a `for_loop`. For example,  
three applications of $R_x(\pi/4)$ yield a net rotation of $3\pi/4$.

In [5]:
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke


q = QuantumRegister(1, "q")
c = ClassicalRegister(1, "c")
qc_for = QuantumCircuit(q, c, name="for_loop_demo")

theta = np.pi/4
with qc_for.for_loop(range(3)):
    qc_for.rx(theta, q[0])

qc_for.measure(q[0], c[0])
backend = FakeSherbrooke()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
cir_isa = pm.run(qc_for)
sampler = SamplerV2(mode=backend)
job = sampler.run([cir_isa], shots=30)
# tqc_for = pm_run(qc_for)
# res_for = run_and_print(tqc_for)
# print(qc_for.draw(output="text"))

In [6]:
results=job.result()
results[0].data

DataBin(c=BitArray(<shape=(), num_shots=30, num_bits=1>))

**Exercise**  
Change `range(3)` to `range(4)` with the same $\theta=\pi/4$. What outcome do you  
expect for the measurement? Verify.

### 3.1.3  `while_loop` — repeat until classical condition flips  

**Docs**  
- https://quantum.cloud.ibm.com/docs/en/guides/classical-feedforward-and-control-flow  

**Idea**  
Initialize $q_0=\lvert 0\rangle$. While classical bit $c_0=0$, apply $X$ then  
re-measure. This will flip $q_0$ once then exit.

In [7]:
q = QuantumRegister(1, "q")
c = ClassicalRegister(1, "c")
qc_while = QuantumCircuit(q, c, name="while_loop_demo")

qc_while.reset(q[0])
qc_while.measure(q[0], c[0])

with qc_while.while_loop((c[0], 0)):
    qc_while.x(q[0])
    qc_while.measure(q[0], c[0])

tqc_while = pm_run(qc_while)
res_while = run_and_print(tqc_while)
print(qc_while.draw(output="text"))

TranspilerError: "The control-flow construct 'while_loop' is not supported by the backend."

**Exercise**  
Insert `h(q[0])` before the first measurement and reason about the number of loop  
iterations. Will it always terminate in one pass? Explain after running.

### 3.1.4  Multiple Choice Questions (5 items)  

**Q1.** In `if_test((c0, 1))`, what is checked at runtime?  
A. The quantum state of qubit 0 equals $\lvert 1\rangle$.  
B. The classical bit `c0` stores value 1.  
C. The last operation on qubit 0 was `x`.  
D. The circuit depth exceeds 1.  

**Q2.** Which snippet best implements *two-way* branching based on `c0`?  
A. `with qc.if_test((c0,1)): qc.x(0)`  
B. `with qc.if_test((c0,1)) as else_: qc.x(0); with else_: qc.z(0)`  
C. `with qc.for_loop(range(2)): qc.x(0)`  
D. `with qc.while_loop((c0,1)): qc.x(0)`  

**Q3.** Consider the loop below with $\theta=\pi/6$:  
```python
with qc.for_loop(range(6)):
    qc.rx(theta, 0)
```  
What is the net rotation about $X$ (mod $2\pi$)?  
A. $\pi/6$  
B. $\pi/2$  
C. $\pi$  
D. $2\pi$  

**Q4.** Which statement about `while_loop` is most accurate?  
A. It is unrolled at transpile-time to a fixed number of iterations.  
B. It evaluates a classical condition during execution and may terminate early.  
C. It requires an `if_else` inside to terminate.  
D. It cannot include measurements.  

**Q5.** You must transpile before `run`. Which approach fits the Study Guide  
best for this notebook?  
A. `tqc = transpile(qc, basis_gates=['cx','rz'])`  
B. `pm = generate_preset_pass_manager(backend); tqc = pm.run(qc)`  
C. `tqc = qc` because primitives transpile internally.  
D. No transpilation is needed on fake backends.  

<details>
<summary><b>Answer Key</b></summary>

- Q1: **B**  
- Q2: **B**  
- Q3: **C** (six times $\pi/6$ equals $\pi$)  
- Q4: **B**  
- Q5: **B**  

</details>